# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
%matplotlib inline

## Read in an Image

In [77]:
#reading in an image
#image = mpimg.imread('test_images/solidWhiteRight.jpg')
#image = mpimg.imread('test.jpg')
#image = mpimg.imread('test_images/solidWhiteCurve.jpg')
#image = mpimg.imread('test_images/solidYellowCurve.jpg')
#image = mpimg.imread('test_images/solidYellowCurve2.jpg')
#image = mpimg.imread('test_images/solidYellowLeft.jpg')
#image = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
#print('This image is:', type(image), 'with dimensions:', image.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [80]:
import math
import glob

def img_read(path):
    img_set=[]
    name_set=[]
    for img in glob.glob(path):    
        file = open(img, 'r')
        file_name = os.path.basename(file.name) [:-4]  + "_output"
        file.close()
        n=mpimg.imread(img)
        img_set.append(n)
        name_set.append(file_name)
    return (img_set,name_set)
        



def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny_edges(img, low_threshold, high_threshold):
#     """Applies the Canny transform"""
     return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
#     """Applies a Gaussian Noise kernel"""
     return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
#     """ Applies an image mask """"
        #defining a blank mask to start with
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255  
        # filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
        #  returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap):
    #create a blank (white) copy with the same size
    line_image = np.copy(image)*0
    line_image_r = np.copy(image)*0
    line_image_l= np.copy(image)*0

    #Run Hough

    lines = cv2. HoughLinesP(masked_edges, rho, theta, treshold, np.array([]), min_line_len, max_line_gap)


    # Draw segments

    #for segment in lines:
        #for x1,y1,x2,y2 in segment:
             #if (abs(y1-y2) > 10) and (y1-y2)< -3: 
                 #cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    x_r=[]
    y_r=[]
    x_l=[]
    y_l=[]
    z_right=[]
    z_left=[]
    x1_new=[]
    y1_new=[]
    x2_new=[]
    y2_new=[]


    for segment in lines:
        for x1,y1,x2,y2 in segment:
                # ignoring the horizontal lines and separating left and right
            if (abs(y1-y2) > 10) and (y1-y2)<-3: 
                x_r+=[x1,x2]
                y_r+=[y1,y2]
            elif (abs(y1-y2) > 10) and (y1-y2)> 3:
                x_l+=[x1,x2]
                y_l+=[y1,y2]


    #Right Lane Border
    z_right=np.polyfit(x_r,y_r,1)
    m_r,b_r=z_right
    x1_r= 550
    y1_r= int((x1_r) * m_r + b_r)
    x2_r= 820
    y2_r= int((x2_r) * m_r + b_r)
    cv2.line(line_image_r,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),10)

    #Left Lane Border
    z_left=np.polyfit(x_l,y_l,1)
    m_l,b_l=z_left
    x1_l= 220
    y1_l= int((x1_l) * m_l + b_l)
    x2_l= 430
    y2_l= int((x2_l) * m_l + b_l)

    cv2.line(line_image_l,(x1_l,y1_l),(x2_l,y2_l),(255,0,0),10)

    return cv2.addWeighted(line_image_l, 1, line_image_r,1,0)  



def weighted_img(initial_img,a,img, b, c):
#     """
#     `img` is the output of the hough_lines(), An image with lines drawn on it.
#     Should be a blank image (all black) with lines drawn on it.
    
#     `initial_img` should be the image before any processing.
    
#     The result image is computed as follows:
    
#     initial_img * α + img * β + γ
#     NOTE: initial_img and img must be the same shape!
#     """
     return cv2.addWeighted(initial_img,a,img, b, c)

def write_image(img,path,name): 
    
    imageio.imwrite(path + name +'.jpg', img)



## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [82]:
import os
import imageio

#Variables

##Paths
input_path = 'test_images/*.jpg'
output_path = 'test_images_output/'

##Canny
lower_treshold = 50
higher_treshold = 100

##Gauss
kernel_size=13

##Region of Interest
imshape =image.shape
vertices = np.array([[(0.1*imshape[1],0.9*imshape[0]),(0.4*imshape[1],0.6*imshape[0]),(0.6*imshape[1],0.6*imshape[0]),(0.9*imshape[1],0.9*imshape[0])]],dtype=np.int32)

## Hough Parameters
rho = 1
theta = np.pi/180
treshold = 1
min_line_len = 10
max_line_gap = 5


#Pipeline

image_set = img_read(input_path)

index=0

for x in image_set[0]:
    
    gray = grayscale(image_set[0][index])

    gauss = gaussian_blur(gray, kernel_size)

    canny = canny_edges(gauss,lower_treshold,higher_treshold)

    region = region_of_interest(canny,vertices)

    hough_extrap = draw_lines(region, rho, theta, treshold, min_line_len, max_line_gap)

    orig_w_lane_lines = weighted_img(hough_extrap,0.8,image_set[0][index],1,0)
    
    write_image(orig_w_lane_lines, output_path,image_set[1][index])
    
    index+=1

 

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [83]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [84]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [85]:
def process_image(image):
    
    #Filter Parameters
    kernel_size=9

    #Canny Parameters
    lower_treshold =50
    higher_treshold =200
    ignore_mask_color = 255
    
    #Hough parameters

    rho = 1
    theta = (np.pi/180)
    treshold = 3
    min_line_length = 20
    max_line_gap = 5
    
    #Pipeline
    
    working_copy = np.copy(image)
    vid_gray = cv2.cvtColor(working_copy,cv2.COLOR_RGB2GRAY) 
    vid_blur_gray = cv2.GaussianBlur(vid_gray,(kernel_size,kernel_size),0)
    #Canny
    vid_edges = cv2.Canny(vid_blur_gray,lower_treshold,higher_treshold)
    
    #Masked Area
    vid_mask = np.zeros_like(vid_edges)
    
    vid_imshape = image.shape
    vid_vertices = np.array([[(0.1*vid_imshape[1],0.9*vid_imshape[0]),(0.45*vid_imshape[1],0.65*vid_imshape[0]),(0.55*vid_imshape[1],0.65*vid_imshape[0]),(0.9*vid_imshape[1],0.9*vid_imshape[0])]],dtype=np.int32)
    cv2.fillPoly(vid_mask, vid_vertices,ignore_mask_color)
    vid_masked_edges = cv2.bitwise_and(vid_edges, vid_mask)

    
    #Hough Transform
    
    #create a blank (white) copy with the same size
    vid_line_image = np.copy(image)*0
    line_image_r = np.copy(image)*0
    line_image_l= np.copy(image)*0
    
    #Run Hough

    vid_lines = cv2. HoughLinesP(vid_masked_edges, rho, theta, treshold, np.array([]), min_line_length, max_line_gap)
    #print ('vid',vid_lines.shape)

    # Draw segments

    #for segment in lines:
    #for x1,y1,x2,y2 in segment:
         #if (abs(y1-y2) > 10) and (y1-y2)< -3: 
             #cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    x_r=[]
    y_r=[]
    x_l=[]
    y_l=[]
    z_right=[]
    z_left=[]
    x1_new=[]
    y1_new=[]
    x2_new=[]
    y2_new=[]
    
    
    if vid_lines is not None:
        for segment in vid_lines:
            for x1,y1,x2,y2 in segment:
                    # ignoring the horizontal lines and separating left and right
                if (abs(y1-y2) > 10) and (y1-y2)<-2: 
                    x_r+=[x1,x2]
                    y_r+=[y1,y2]
                elif (abs(y1-y2) > 10) and (y1-y2)> 2:
                    x_l+=[x1,x2]
                    y_l+=[y1,y2]    
            
            
    #Right Lane Border
    if x_r :    
        z_right=np.polyfit(x_r,y_r,1)
        m_r,b_r=z_right
        x1_r= int(0.6*(vid_imshape[1]))
        y1_r= int((x1_r) * m_r + b_r)
        x2_r= int(0.8*(vid_imshape[1]))
        y2_r= int((x2_r) * m_r + b_r)
        cv2.line(line_image_r,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),10)
    
    #Left Lane Border
    if x_l :
        z_left=np.polyfit(x_l,y_l,1)
        m_l,b_l=z_left
        x1_l= int(0.2*(vid_imshape[1]))
        y1_l= int((x1_l) * m_l + b_l)
        x2_l= int(0.4*(vid_imshape[1]))
        y2_l= int((x2_l) * m_l + b_l)
        cv2.line(line_image_l,(x1_l,y1_l),(x2_l,y2_l),(255,0,0),10)

    vid_line_image = cv2.addWeighted(line_image_l, 1, line_image_r,1,0) 
                

    # color_edges

    vid_color_edges = np.dstack((vid_edges,vid_edges,vid_edges))

    #Draw the lines on the line_image
    print(vid_color_edges.shape)
    print(vid_line_image.shape)
    vid_line_edges = cv2.addWeighted(vid_color_edges, 0.1, vid_line_image,1,0)
    
    #Show Masked Area
    vid_color_mask = np.dstack((vid_mask,vid_mask,vid_mask))

    vid_orig_w_polygon = cv2.addWeighted(image, 0.8, vid_color_mask,0.1,0)
    
    
    return cv2.addWeighted(vid_orig_w_polygon,1, vid_line_edges,1,0)
    
    
   

Let's try the one with the solid white lane on the right first ...

In [86]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

(540, 960, 3)
(540, 960, 3)
[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


  0%|                                                  | 0/222 [00:00<?, ?it/s]

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


  2%|▊                                         | 4/222 [00:00<00:06, 31.50it/s]

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


...

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


 99%|███████████████████████████████████████▋| 220/222 [00:06<00:00, 35.29it/s]

(540, 960, 3)
(540, 960, 3)


100%|███████████████████████████████████████▊| 221/222 [00:06<00:00, 33.98it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 6.84 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [87]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [88]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

(540, 960, 3)
(540, 960, 3)
[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


  0%|                                                  | 0/682 [00:00<?, ?it/s]

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


  1%|▏                                         | 4/682 [00:00<00:20, 33.06it/s]

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


...

(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)
(540, 960, 3)


100%|███████████████████████████████████████▉| 680/682 [00:20<00:00, 33.38it/s]

(540, 960, 3)
(540, 960, 3)


100%|███████████████████████████████████████▉| 681/682 [00:20<00:00, 33.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 20.5 s


In [89]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [90]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

(720, 1280, 3)
(720, 1280, 3)
[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


  0%|                                                  | 0/251 [00:00<?, ?it/s]

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


  1%|▌                                         | 3/251 [00:00<00:12, 20.27it/s]

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


...

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


 99%|███████████████████████████████████████▋| 249/251 [00:13<00:00, 18.46it/s]

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


100%|████████████████████████████████████████| 251/251 [00:13<00:00, 18.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 13.8 s


In [91]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))